In [1]:
import pandas as pd
import os
import numpy as np
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
import lightgbm as lgb
import xgboost as xgb

import gc
from numba import jit

random_seed = 33

train_file = 'train_0102_filter.csv'    
write_path = './BayesOpt_result/'

# model_type = 'catboost'
model_type = 'lgb'
# model_type = 'xgb'

if model_type == 'catboost':
    record_file = 'catboost_result.txt'
elif model_type == 'lgb':
    record_file = 'lgb_result.txt'
elif model_type == 'xgb':
    record_file = 'xgb_result.txt'
else:
    pass

iteration = 150
init_it = 50
category_list = []


In [2]:
@jit
def qwk(a1, a2):
    """
    Source: https://www.kaggle.com/c/data-science-bowl-2019/discussion/114133#latest-660168

    :param a1:
    :param a2:
    :param max_rat:
    :return:
    """
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)

    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))

    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)

    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)

    e = e / a1.shape[0]

    return 1 - o / e

def eval_qwk_lgb_regr(y_true, y_pred):
    """
    Fast cappa eval function for lgb.
    """
    y_pred[y_pred <= 1.12232214] = 0
    y_pred[np.where(np.logical_and(y_pred > 1.12232214, y_pred <= 1.73925866))] = 1
    y_pred[np.where(np.logical_and(y_pred > 1.73925866, y_pred <= 2.22506454))] = 2
    y_pred[y_pred > 2.22506454] = 3

    # y_pred = y_pred.reshape(len(np.unique(y_true)), -1).argmax(axis=0)

    return 'cappa', qwk(y_true, y_pred), True


In [3]:
train = pd.read_csv('../data/preprocess/'+train_file)
cols_to_drop = ['game_session', 'installation_id', 'timestamp', 'accuracy_group', 'timestampDate']
y = train['accuracy_group'].copy()
for c in cols_to_drop:
    if c in train.columns:
        train = train.drop(columns = c)
X = train

## 驗證集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=random_seed, stratify=y)
     
categorical_features_indices = np.where(X_train.columns.isin(category_list))[0]
print(X_train.dtypes[categorical_features_indices])

## catboost 調參
## https://catboost.ai/docs/concepts/loss-functions-regression.html
param_cat={
    'loss_function':'RMSE', ##MAE
    'eval_metric':'RMSE',
    
    'iterations':14000,
    'random_seed':random_seed,
    'thread_count':5,
    'task_type':"GPU",
    'devices':'0:1',
#     'boosting_type':'Ordered',
    'learning_rate':0.03,
    'l2_leaf_reg':20,#20
    'depth':7,
    'bagging_temperature':0.3,
    'random_strength':10,
    # 'rsm':0.8,

    # 'fold_permutation_block':1,
    # 'feature_border_type':'MinEntropy',
    # 'boosting_type':'Ordered',
    # 'leaf_estimation_backtracking':'Armijo',
    
    'one_hot_max_size':200,
#     'grow_policy':'Lossguide',
#     'grow_policy':'Lossguide',
}
param_range_cat={
#     'depth':(5,16.9),
#     'max_leaves':(20,45),
    'depth':(5,12.9),
    'max_leaves':(31,31.5),
    'l2_leaf_reg':(1,100),
    'bagging_temperature':(0.01,5)  
}

## LGB調參
## https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html#lightgbm.LGBMRegressor
## https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

param_lgb = {'n_estimators':10000,
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'subsample': 0.75,
            'subsample_freq': 1,
            'learning_rate': 0.04,
            'feature_fraction': 0.9,
            'max_depth': 15,
            'lambda_l1': 1,  
            'lambda_l2': 1,
            'verbose': 100,
            'early_stopping_rounds': 100, 'eval_metric': 'cappa'
            }

param_range_lgb={
    'max_depth': (5,16),
    'num_leaves': (15,100),
    'feature_fraction': (0.3,1),
    
    'subsample':(0.2,1),
    'subsample_freq':(1,3),
    'lambda_l1':(0.1,50),
    'lambda_l2':(0.1,50),
}


param_clf = {}
param_range = {}

if model_type == 'catboost':
    param_clf = param_cat
    param_range = param_range_cat
elif model_type == 'lgb':
    param_clf = param_lgb
    param_range = param_range_lgb
elif model_type == 'xgb':
    param_clf = param_lgb
    param_range = param_range_lgb
else:
    pass

def bys_train_catboost(depth,max_leaves,l2_leaf_reg,bagging_temperature):
    param_clf['depth']=int(depth)
    param_clf['max_leaves']=int(max_leaves)
    param_clf['l2_leaf_reg']=l2_leaf_reg
    param_clf['bagging_temperature']=bagging_temperature
    
    model = CatBoostRegressor(**param_clf)
    model.fit(X_train, y_train,
    cat_features=categorical_features_indices,    
    eval_set=(X_test, y_test),
    early_stopping_rounds=200,
    verbose=50) 
     
    score_max = model.get_best_score()['validation']['RMSE']
    
    print(int(depth),int(max_leaves),l2_leaf_reg,bagging_temperature)
    print(score_max)
    
    with open(write_path+record_file,'a') as f:
        print('depth',int(depth),file=f)
        print('max_leaves',int(max_leaves),file=f)
        print('l2_leaf_reg',l2_leaf_reg,file=f)
        print('bagging_temperature',bagging_temperature,file=f)
        print(score_max,file=f)
        print('',file=f)
    
    return -score_max

def bys_train_lgb(max_depth,num_leaves,feature_fraction,subsample,subsample_freq,lambda_l1,lambda_l2):
    param_clf['max_depth']=int(max_depth)
    param_clf['num_leaves']=int(num_leaves)
    param_clf['feature_fraction']=feature_fraction
    param_clf['subsample']=subsample
    param_clf['subsample_freq']=int(subsample_freq)
    param_clf['lambda_l1']=lambda_l1
    param_clf['lambda_l2']=lambda_l2
        
    model = lgb.LGBMRegressor(**param_clf)
    model.fit(X=X_train, y=y_train,
              eval_set=[(X_train, y_train),(X_test, y_test)], eval_metric=eval_qwk_lgb_regr,
              verbose=param_clf['verbose'], early_stopping_rounds=param_clf['early_stopping_rounds'],
              categorical_feature=categorical_features_indices)

    score_max = model.best_score_['valid_1']['rmse']
    
    print("parameter",int(max_depth),int(num_leaves),feature_fraction,subsample,int(subsample_freq),lambda_l1,lambda_l2)
    print("score",score_max)
    
    with open(write_path+record_file,'a') as f:
        print('max_depth',int(max_depth),file=f)
        print('num_leaves',int(num_leaves),file=f)
        print('feature_fraction',feature_fraction,file=f)
        print('subsample',subsample,file=f)
        print('subsample_freq',int(subsample_freq),file=f)
        print('lambda_l1',lambda_l1,file=f)
        print('lambda_l2',lambda_l2,file=f)
        print(score_max,file=f)
        print('',file=f)
    
    return -score_max

def main(write_path='./BayesOpt_result/',record_file='Bayes_result.txt',iteration=5,init_it=5):
    print(iteration)
    with open(write_path+record_file,'a') as f:
        print('\n{}'.format(train_file),file=f)
        
    if model_type == 'catboost':
        Bys_opt = BayesianOptimization(bys_train_catboost,param_range)#, acq="ucb", kappa=1) 
    elif model_type == 'lgb':
        Bys_opt = BayesianOptimization(bys_train_lgb,param_range)#, acq="ucb", kappa=1) 
    elif model_type == 'xgb':
        Bys_opt = BayesianOptimization(bys_train_lgb,param_range)#, acq="ucb", kappaand=1) 
    else:
        pass
    
    Bys_opt.maximize(n_iter=iteration, init_points=init_it)
    print(Bys_opt.max)
    
    with open(write_path+record_file,'a') as f:
        print('Max para',Bys_opt.max,file=f)

main(write_path=write_path, record_file=record_file, iteration=iteration,init_it=init_it)


Series([], dtype: object)
150
|   iter    |  target   | featur... | lambda_l1 | lambda_l2 | max_depth | num_le... | subsample | subsam... |
-------------------------------------------------------------------------------------------------------------


/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/basic.py:1236: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.categorical_feature == categorical_feature:
/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/basic.py:1242: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif categorical_feature == 'auto':
/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/

Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 1.00672	training's cappa: 0.576136	valid_1's rmse: 1.03153	valid_1's cappa: 0.546042
[200]	training's rmse: 0.970168	training's cappa: 0.609252	valid_1's rmse: 1.00979	valid_1's cappa: 0.576004
[300]	training's rmse: 0.953562	training's cappa: 0.626079	valid_1's rmse: 1.00532	valid_1's cappa: 0.571575
Early stopping, best iteration is:
[200]	training's rmse: 0.970168	training's cappa: 0.609252	valid_1's rmse: 1.00979	valid_1's cappa: 0.576004
parameter 7 31 0.39875586863841606 0.20261045664943975 2 30.383146004164576 31.10485162007598
score 1.0097852578460367
|  1        | -1.01     |  0.3988   |  30.38    |  31.1     |  7.757    |  31.46    |  0.2026   |  2.789    |


/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/basic.py:1236: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.categorical_feature == categorical_feature:
/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/basic.py:1242: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif categorical_feature == 'auto':
/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.87892	training's cappa: 0.702592	valid_1's rmse: 0.995021	valid_1's cappa: 0.579922
[200]	training's rmse: 0.795531	training's cappa: 0.768347	valid_1's rmse: 0.991196	valid_1's cappa: 0.574621
Early stopping, best iteration is:
[106]	training's rmse: 0.872242	training's cappa: 0.706366	valid_1's rmse: 0.994548	valid_1's cappa: 0.583846
parameter 12 62 0.5427973204105535 0.5480450734352034 2 6.689119978504311 20.936995584523917
score 0.9945479655539644
|  2        | -0.9945   |  0.5428   |  6.689    |  20.94    |  12.25    |  62.34    |  0.548    |  2.436    |
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.97243	training's cappa: 0.609855	valid_1's rmse: 1.01062	valid_1's cappa: 0.565982
[200]	training's rmse: 0.934723	training's cappa: 0.646126	valid_1's rmse: 1.00147	valid_1's cappa: 0.57125
[300]	training's rmse: 0.912785	training's cappa: 0.667863	valid_1's

...

Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.929084	training's cappa: 0.650398	valid_1's rmse: 0.993206	valid_1's cappa: 0.579355
[200]	training's rmse: 0.877027	training's cappa: 0.694018	valid_1's rmse: 0.987544	valid_1's cappa: 0.585661
[300]	training's rmse: 0.840084	training's cappa: 0.72957	valid_1's rmse: 0.987333	valid_1's cappa: 0.584354
Early stopping, best iteration is:
[211]	training's rmse: 0.872074	training's cappa: 0.698852	valid_1's rmse: 0.986824	valid_1's cappa: 0.588755
parameter 14 25 0.8992201232752219 0.9931225042005585 2 16.902534320544287 0.5910489401840201
score 0.9868242304375798
|  198      | -0.9868   |  0.8992   |  16.9     |  0.591    |  14.25    |  25.88    |  0.9931   |  2.936    |


/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/basic.py:1236: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.categorical_feature == categorical_feature:
/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/basic.py:1242: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif categorical_feature == 'auto':
/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/

Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.919635	training's cappa: 0.663645	valid_1's rmse: 0.999004	valid_1's cappa: 0.572415
[200]	training's rmse: 0.865206	training's cappa: 0.708036	valid_1's rmse: 0.994129	valid_1's cappa: 0.577398
[300]	training's rmse: 0.835892	training's cappa: 0.733076	valid_1's rmse: 0.993747	valid_1's cappa: 0.579375
Early stopping, best iteration is:
[282]	training's rmse: 0.839856	training's cappa: 0.730213	valid_1's rmse: 0.993638	valid_1's cappa: 0.580784
parameter 11 99 0.9943637536675567 0.9181390846767743 2 39.637286808419425 1.4450322190814164
score 0.9936376523993588
|  199      | -0.9936   |  0.9944   |  39.64    |  1.445    |  11.05    |  99.6     |  0.9181   |  2.992    |


/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/basic.py:1236: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.categorical_feature == categorical_feature:
/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/basic.py:1242: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif categorical_feature == 'auto':
/home/jupyter-shliu/.local/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/

Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.935631	training's cappa: 0.644252	valid_1's rmse: 0.994617	valid_1's cappa: 0.578656
[200]	training's rmse: 0.887007	training's cappa: 0.686718	valid_1's rmse: 0.989256	valid_1's cappa: 0.582767
[300]	training's rmse: 0.853846	training's cappa: 0.716807	valid_1's rmse: 0.988911	valid_1's cappa: 0.586038
Early stopping, best iteration is:
[293]	training's rmse: 0.855943	training's cappa: 0.715371	valid_1's rmse: 0.988981	valid_1's cappa: 0.588265
parameter 13 25 0.8086279281630093 0.997751036426046 2 22.2755144322752 0.5934254234483143
score 0.988980997431866
|  200      | -0.989    |  0.8086   |  22.28    |  0.5934   |  13.13    |  25.55    |  0.9978   |  2.855    |
{'target': -0.9868242304375798, 'params': {'feature_fraction': 0.8992201232752219, 'lambda_l1': 16.902534320544287, 'lambda_l2': 0.5910489401840201, 'max_depth': 14.251454256504196, 'num_leaves': 25.88298375466983, 'subsample': 0.99312250

## 原理理解
## https://github.com/fmfn/BayesianOptimization/blob/master/examples/visualization.ipynb


#### init step越多，越不容易卡在local mini
#### Kappa決定confidence interval範圍
#### 不了解kappa, GP的過程 唉